In [1]:
import pandas as pd

df = pd.read_csv('prabowo_3month_labeled.csv')
df.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,label
0,1906370082558890432,Sun Mar 30 23:56:34 +0000 2025,92,@bujas__ @hrdbacot dan ini keberhasilan presid...,1906495800852758702,NaN,bjs________,in,NaN,0,0,0,https://x.com/JBjamesbond07/status/19064958008...,1759273445500850176,JBjamesbond07,negatif
1,1906477277870989601,Sun Mar 30 22:42:58 +0000 2025,74,Rocky Gerung Bongkar Borok Tim Komunikasi Prab...,1906477277870989601,NaN,NaN,in,NaN,1,2,36,https://x.com/OposisiCerdas/status/19064772778...,887743587579944960,OposisiCerdas,negatif
2,1906367746746859858,Sun Mar 30 15:27:44 +0000 2025,127,Ijin jend @prabowo Bang @sjafriesjams Akun yg ...,1906367746746859858,NaN,NaN,in,NaN,0,5,68,https://x.com/Minang_cyber/status/190636774674...,1350917154,Minang_cyber,negatif
3,1905980742347505935,Sun Mar 30 07:42:03 +0000 2025,81,@NenkMonica @erickthohir prabowo bener2 sudah ...,1906250555070231023,NaN,NenkMonica,in,NaN,1,1,14,https://x.com/opinimanweb/status/1906250555070...,1073590382,opinimanweb,negatif
4,1906219524816540145,Sun Mar 30 05:38:45 +0000 2025,291,Prabowo Undang Presiden-Wapres Terdahulu ke Op...,1906219524816540145,NaN,NaN,in,NaN,20,55,27,https://x.com/CNNIndonesia/status/190621952481...,17128975,CNNIndonesia,netral


In [2]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Inisialisasi stemmer dan stopwords
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

# Mention penting yang ingin disimpan (whitelist)
mention_whitelist = ['prabowo']

# Kamus normalisasi kata informal → formal
normalisasi_kata = {
    'gak': 'tidak',
    'ga': 'tidak',
    'yg': 'yang',
    'utk': 'untuk',
    'dgn': 'dengan',
    'apik': 'baik',
    'sdh': 'sudah',
    'krn': 'karena',
    'apik': 'bagus',
    'tdk': 'tidak',
    'klo': 'kalo',
    'sbg': 'sebagai',
    'gue': 'aku',
    'dlm': 'dalam',
    'jgn': 'jangan',
    'jkw': 'jokowi',
    'org': 'orang',
    'nggak': 'tidak',
    'aja': '',
    'amp': '',
    'nya': '',
    'ya': '',
    'gitu': '',
    'loh': '',
    'dong': '',
    'sih': '',
    'deh': '',
    'nih': '',
    'kok': '',
}

def preprocessing(text):
    # 2. Tangani mention: simpan jika ada di whitelist, hapus lainnya
    text = re.sub(r'@(\w+)', lambda m: m.group() if m.group(1).lower() in mention_whitelist else '', text)

    text = re.sub(r'http\S+|www\S+|<.*?>|#', ' ', text)
    
    # 4. Hapus tag HTML (jika ada)
    text = re.sub(r'<.*?>', '', text)

    # 5. Hapus karakter non-huruf (angka, simbol)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # 6. Lowercase
    text = text.lower()

    # 7. Tokenisasi
    tokens = word_tokenize(text)

    # 8. Normalisasi kata informal
    tokens = [normalisasi_kata.get(word, word) for word in tokens]

    # 9. Hapus stopwords dan kata kosong/panjang pendek
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2 and word != '']

    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    
    # 11. Gabung kembali jadi string
    return ' '.join(tokens)


In [3]:
import swifter

df['text_processed'] = df['full_text'].swifter.apply(preprocessing)
df.head()

Pandas Apply:   0%|          | 0/927 [00:00<?, ?it/s]

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,label,text_processed
0,1906370082558890432,Sun Mar 30 23:56:34 +0000 2025,92,@bujas__ @hrdbacot dan ini keberhasilan presid...,1906495800852758702,NaN,bjs________,in,NaN,0,0,0,https://x.com/JBjamesbond07/status/19064958008...,1759273445500850176,JBjamesbond07,negatif,hasil presiden jokowi lanjut presiden prabowo ...
1,1906477277870989601,Sun Mar 30 22:42:58 +0000 2025,74,Rocky Gerung Bongkar Borok Tim Komunikasi Prab...,1906477277870989601,NaN,NaN,in,NaN,1,2,36,https://x.com/OposisiCerdas/status/19064772778...,887743587579944960,OposisiCerdas,negatif,rocky gerung bongkar borok tim komunikasi prab...
2,1906367746746859858,Sun Mar 30 15:27:44 +0000 2025,127,Ijin jend @prabowo Bang @sjafriesjams Akun yg ...,1906367746746859858,NaN,NaN,in,NaN,0,5,68,https://x.com/Minang_cyber/status/190636774674...,1350917154,Minang_cyber,negatif,ijin jend prabowo bang akun hina prabowo dri m...
3,1905980742347505935,Sun Mar 30 07:42:03 +0000 2025,81,@NenkMonica @erickthohir prabowo bener2 sudah ...,1906250555070231023,NaN,NenkMonica,in,NaN,1,1,14,https://x.com/opinimanweb/status/1906250555070...,1073590382,opinimanweb,negatif,prabowo bener nilai mata bantu arah abai presi...
4,1906219524816540145,Sun Mar 30 05:38:45 +0000 2025,291,Prabowo Undang Presiden-Wapres Terdahulu ke Op...,1906219524816540145,NaN,NaN,in,NaN,20,55,27,https://x.com/CNNIndonesia/status/190621952481...,17128975,CNNIndonesia,netral,prabowo undang presiden wapres open house ista...


In [5]:
from collections import Counter
import re

df_copy = df.copy()

# Gabungkan semua teks jadi satu string
text = ' '.join(df_copy['text_processed'].astype(str))

# Preprocessing: lowercase + tokenisasi
words = re.findall(r'\b\w+\b', text.lower())

# Hitung frekuensi kata
freq = Counter(words)

# Ambil 50 kata paling sering
top_50 = freq.most_common(150)

# Cetak hasil
for word, count in top_50:
    print(f"{word}: {count}")


presiden: 1101
prabowo: 1067
jokowi: 193
rakyat: 184
indonesia: 119
subianto: 95
negara: 88
orang: 86
jabat: 83
perintah: 82
menteri: 75
anak: 62
kalo: 59
tni: 59
pilih: 59
bijak: 59
udah: 49
dukung: 48
kepala: 42
kuasa: 36
turun: 35
program: 34
pimpin: 33
kali: 33
bikin: 33
beliau: 33
utus: 33
makan: 32
bilang: 32
menang: 32
mantan: 32
partai: 31
gibran: 31
masyarakat: 31
ketua: 31
hasil: 30
kabinet: 30
khusus: 30
kerja: 29
langsung: 29
demo: 28
tuju: 28
daerah: 28
politik: 27
salah: 27
wakil: 27
dunia: 26
biar: 26
uang: 26
mulyono: 26
bpk: 26
miftah: 26
bantu: 25
berani: 25
korupsi: 25
kampanye: 25
anggar: 24
gaji: 24
bangsa: 23
kasih: 23
dpr: 23
sby: 23
nama: 23
tuh: 23
tolong: 23
publik: 23
banget: 23
anjing: 22
hukum: 22
kuat: 22
emang: 22
tugas: 22
kaya: 22
bener: 21
wapres: 21
polri: 21
adil: 21
malu: 21
jalan: 21
milih: 21
ganti: 20
gin: 20
bawa: 20
suara: 20
takut: 20
negeri: 20
danantara: 20
pdip: 20
ndasmu: 20
yth: 20
harap: 20
komunikasi: 19
hina: 19
laku: 19
bawah: 19
hati

In [6]:
df.to_csv('prabowo_3month_labeled.csv', index=False)